### 4 RAG (Retrieval Augmented Genative)

#### 4.03 Load API Key

In [1]:
from dotenv import load_dotenv

load_dotenv() # Loads .env file

True

#### 4.04 Install Requirements

In [2]:
!pip install langchain --quiet          # LangChain: LangChain-Core for based library
!pip install python-dotenv --quiet      # Load Key
!pip install streamlit --quiet          # GUI
#!pip install unstructured --quiet       # Python Unstructure text library 
!pip install csvLoader --quiet       # Python Unstructure text library 
!pip install tiktoken --quiet           # python text token parser
!pip install faiss-cpu --quiet          # Facebook AI Similarity Search Library

!pip install google-generativeai --quiet    # Google LLM (Gemini Model for Generative AI) 
!pip install langchain_google_genai --quiet # LangChain for Google LLM (Gemini Model)

# > pip install -U langchain-community
!pip install langchain-community --quiet     # LangChain tools for third party LLM Integration

#### 4.05 Import Library

In [3]:
import streamlit as st
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import time
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
#from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
#from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.question_answering import load_qa_chain
import google.generativeai as genai
from langchain.prompts import PromptTemplate
# from langchain.document_loaders import UnstructuredURLLoader # deprecated
#from langchain_community.document_loaders import UnstructuredURLLoader
from langchain.document_loaders.csv_loader import CSVLoader

#### 4.06 Configure Gemini Model

In [4]:
genai.configure(api_key=os.getenv("GOOGLE_API_KEY")) # Loads API key

#### 4.07 Get Conversational Chain

In [5]:
def get_conversational_chain():
    '''
    Question and Answer Chain
    '''
    # Define a prompt template for asking questions based on a given context
    prompt_template = """
    Answer the question as detailed as possible from the provided context, 
    make sure to provide all the details,
    if the answer is not in the provided context just say, 
    "answer is not available in the context", don't provide the wrong answer\n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """

    # Initialize a ChatGoogleGenerativeAI model for conversational AI
    model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3)

    # Create a prompt template with input variables "context" and "question"
    prompt = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )

    # Load a question-answering chain with the specified model and prompt
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

    return chain

#### 4.08 Load CSV into Texts

In [7]:
def get_text_from_csv (csv_files = ["q_a_pair.csv"]):
    # Load data from FAQ sheet
    #loader = CSVLoader(file_path='codebasics_faqs.csv', source_column="prompt")
    #data = loader.load()
    q_a_texts = []
    for csv_file in csv_files:
        print('cvs_file.name:', csv_file.name)
        loader = CSVLoader(file_path=csv_file.name, source_column="prompt")
        q_a_texts.append(loader.load())
    return q_a_texts

#### 4.09 Texts into Chunks

In [8]:
def get_chunks(q_a_texts):
    '''
    The RecursiveCharacterTextSplitter takes a large text 
    and splits it based on a specified chunk size.
    '''
    #text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    #chunks = text_splitter.split_text(text)
    raw_text = " "
    i = 0
    print('len(q_a_texts):', len(q_a_texts))
    for q_a_text in q_a_texts:
        print('q_a_text[i].page_content[:100]:')
        print(q_a_text[i].page_content[:100])
    	# q_a_text:
        raw_text += q_a_text[i].page_content
        i += 1
    
    text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", ".", ",", "\'"],
                                                   chunk_size=1000, chunk_overlap=200)
    #docs = text_splitter.split_documents(data)
    chunks = text_splitter.split_text(raw_text)
    return chunks

#### 4.10 Create Vector DB

In [9]:
def create_vector_db(text_chunks, vectordb_name, embeddings):
    # Load data from FAQ sheet
    #loader = CSVLoader(file_path='q_a_pair.csv', source_column="prompt")
    #data = loader.load()
    # Save vector database locally
    #vectordb.save_local(vectordb_file_path)

    # Create a FAISS instance for vector database from 'data'
    #vectordb = FAISS.from_texts(documents=chunk_text,
    #                                embedding=instructor_embeddings)
    #embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

    #instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large")
    # vectordb = FAISS.from_texts(text_chunks, embedding=embeddings)
    # Create a vector store using FAISS from the provided text chunks and embeddings
    #embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    vectordb = FAISS.from_texts(text_chunks, embedding=embeddings)
    time.sleep(2)
    
    # Save the vector store locally with the name "faiss_index"
    vectordb.save_local(vectordb_name)

#### 4.11 Get Conversational Chain

In [10]:
def get_conversational_chain():
    '''
    Question and Answer Chain
    '''
    # Define a prompt template for asking questions based on a given context
    prompt_template = """
    Answer the question as detailed as possible from the provided context, 
    make sure to provide all the details,
    if the answer is not in the provided context just say, 
    "answer is not available in the context", don't provide the wrong answer\n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """

    # Initialize a ChatGoogleGenerativeAI model for conversational AI
    model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3)

    # Create a prompt template with input variables "context" and "question"
    # https://github.com/EgoAlpha/prompt-in-context-learning/blob/main/PromptEngineering.md
    prompt = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )

    # Load a question-answering chain with the specified model and prompt
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

    return chain

#### 4.12 User Inupt

In [11]:
def user_input(user_question, vectordb_name, instructor_embeddings):
    '''
    Tak user input
    '''
    # Create embeddings for the user question using a Google Generative AI model
    # https://www.reddit.com/r/LocalLLaMA/comments/15a3yef/embeddings_or_instructor_embeddings/
    #instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large")
    #new_db = FAISS.load_local("faiss_index", instructor_embeddings, 
    #                          allow_dangerous_deserialization=True)
    
    # Load a FAISS vector database from a local file
    # Add allow_dangerous_deserialization=True=True to trust external file.
    #embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001"), 
    new_db = FAISS.load_local(vectordb_name, instructor_embeddings, 
                              allow_dangerous_deserialization=True)

    # Perform similarity search in the vector database based on the user question
    docs = new_db.similarity_search(user_question)

    # Obtain a conversational question-answering chain
    chain = get_conversational_chain()

    # Use the conversational chain to get a response based on the user question and retrieved documents
    # We pass in "input_documents" for prompt {context} and "question" for prompt {question}
    response = chain(
        {"input_documents": docs, "question": user_question}, return_only_outputs=True
    )

    # Print the response to the console
    print(response)

    # Display the response in a Streamlit app (assuming 'st' is a Streamlit module)
    st.write("Reply: ", response["output_text"])

#### 4.13 Run Main Program

In [6]:
def main():
    st.set_page_config("RAG with Excel")
    st.header("Chat with RAG (Retrieval Augmented Generitive) using Gemini")

    user_question = st.text_input("Ask a Question from the Excel Files")

    vectordb_name="faiss_index"
    #instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large")
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    if user_question:
        user_input(user_question, vectordb_name, embeddings)

    with st.sidebar:
        st.title("Menu:")
        st.sidebar.title("Excel Files")
        upload_files = st.file_uploader("Upload your Excel Files and Click on the Submit & Process Button", 
                                      accept_multiple_files=True)
        if st.button("Submit & Process"):
            with st.spinner("Processing..."):
                # raw_text = get_pdf_text(pdf_docs)
                #print('upload_file.name:', upload_file.name)
                raw_text = get_text_from_csv (upload_files)
                text_chunks = get_chunks(raw_text)
                create_vector_db(text_chunks, vectordb_name, embeddings)
                st.success("Done")


if __name__ == "__main__":
    main()


2024-06-01 08:00:00.467 
  command:

    streamlit run /home/peter/.local/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-06-01 08:00:00.469 Session state does not function when running a script without `streamlit run`


In [31]:
# > streamlit run main.py